In [4]:
import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.training import Example
from spacy.tokens import DocBin
import pandas as pd
import json
import re
import numpy as np
import os
from datetime import datetime

class SirenExtractor:
    """Classe pour extraire et valider les numéros SIREN depuis un texte"""
    
    def __init__(self, modele_spacy="fr_core_news_md"):
        """Initialiser l'extracteur SIREN"""
        self.nlp = self._configurer_nlp(modele_spacy)
    
    def _creer_patterns_siren(self):
        """Définir les patterns pour la détection des numéros SIREN"""
        patterns = [
            # SIREN sous forme de 9 chiffres consécutifs
            {"label": "SIREN", "pattern": [{"TEXT": {"REGEX": r"^\d{9}$"}}]},
            
            # SIREN sous forme de 3 groupes de 3 chiffres
            {"label": "SIREN", "pattern": [
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": {"REGEX": r"^\d{3}$"}}
            ]},
            
            # SIREN précédé de "RCS"
            {"label": "SIREN", "pattern": [
                {"LOWER": "rcs"},
                {"IS_ALPHA": True, "OP": "?"},
                {"TEXT": {"REGEX": r"^\d{9}$"}}
            ]},
            
            # SIREN précédé de "RCS" avec format 3-3-3
            {"label": "SIREN", "pattern": [
                {"LOWER": "rcs"},
                {"IS_ALPHA": True, "OP": "?"},
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": {"REGEX": r"^\d{3}$"}}
            ]},
            
            # SIREN avec mention explicite
            {"label": "SIREN", "pattern": [
                {"LOWER": {"IN": ["siren", "siret"]}},
                {"TEXT": {"REGEX": r"^\d{9}$"}}
            ]},
            
            # SIREN avec séparateurs
            {"label": "SIREN", "pattern": [
                {"LOWER": {"IN": ["siren", "siret", "rcs"]}},
                {"TEXT": {"IN": [":", "n°", "numéro", "no"]}},
                {"TEXT": {"REGEX": r"^\d{9}$"}}
            ]},
            
            # SIREN avec tirets
            {"label": "SIREN", "pattern": [
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": "-"},
                {"TEXT": {"REGEX": r"^\d{3}$"}},
                {"TEXT": "-"},
                {"TEXT": {"REGEX": r"^\d{3}$"}}
            ]}
        ]
        return patterns
    
    def _configurer_nlp(self, modele_spacy):
        """Configurer le pipeline spaCy avec EntityRuler"""
        try:
            nlp = spacy.load(modele_spacy)
        except OSError:
            print(f"Modèle {modele_spacy} non trouvé. Tentative avec fr_core_news_sm...")
            try:
                nlp = spacy.load("fr_core_news_sm")
            except OSError:
                print("Aucun modèle français trouvé. Utilisation du modèle blank...")
                nlp = spacy.blank("fr")
        
        # Ajouter EntityRuler
        if "entity_ruler" not in nlp.pipe_names:
            ruler = nlp.add_pipe("entity_ruler", before="ner" if "ner" in nlp.pipe_names else "last")
            patterns = self._creer_patterns_siren()
            ruler.add_patterns(patterns)
        
        return nlp
    
    def valider_siren(self, siren_str):
        """Valider un numéro SIREN avec l'algorithme de Luhn"""
        # Nettoyer le SIREN (garder seulement les chiffres)
        siren = re.sub(r'\D', '', str(siren_str))
        
        # Vérifier la longueur et que ce sont des chiffres
        if len(siren) != 9 or not siren.isdigit():
            return False
        
        # Algorithme de Luhn
        total = 0
        for i, digit in enumerate(siren):
            n = int(digit)
            if i % 2 == 1:  # Position paire (en partant de 0)
                n *= 2
                if n > 9:
                    n -= 9
            total += n
        
        return total % 10 == 0
    
    def extraire_numeros_siren_regex(self, texte):
        """Extraire les numéros SIREN avec regex (méthode de fallback)"""
        # Pattern pour différents formats de SIREN
        patterns = [
            r'\b\d{9}\b',  # 9 chiffres consécutifs
            r'\b\d{3}[\s-]\d{3}[\s-]\d{3}\b',  # Format XXX XXX XXX ou XXX-XXX-XXX
            r'\b\d{3}\s+\d{3}\s+\d{3}\b'  # Format avec espaces multiples
        ]
        
        numeros_siren = []
        for pattern in patterns:
            for match in re.finditer(pattern, texte):
                # Nettoyer le numéro
                siren_propre = re.sub(r'\D', '', match.group())
                if len(siren_propre) == 9:
                    numeros_siren.append(siren_propre)
        
        # Supprimer les doublons tout en gardant l'ordre
        return list(dict.fromkeys(numeros_siren))
    
    def extraire_siren_avec_spacy(self, texte):
        """Extraire les SIREN avec spaCy"""
        doc = self.nlp(texte)
        numeros_siren = []
        
        for ent in doc.ents:
            if ent.label_ == "SIREN":
                # Nettoyer le numéro extrait
                siren_propre = re.sub(r'\D', '', ent.text)
                if len(siren_propre) == 9:
                    numeros_siren.append(siren_propre)
        
        return list(dict.fromkeys(numeros_siren))
    
    def extraire_tous_siren(self, texte, valider=True):
        """Extraire tous les SIREN du texte en combinant spaCy et regex"""
        # Extraction avec spaCy
        siren_spacy = self.extraire_siren_avec_spacy(texte)
        
        # Extraction avec regex (fallback)
        siren_regex = self.extraire_numeros_siren_regex(texte)
        
        # Combiner les résultats
        tous_siren = list(dict.fromkeys(siren_spacy + siren_regex))
        
        # Valider si demandé
        if valider:
            siren_valides = [siren for siren in tous_siren if self.valider_siren(siren)]
            siren_invalides = [siren for siren in tous_siren if not self.valider_siren(siren)]
            return siren_valides, siren_invalides
        
        return tous_siren, []
    
    def analyser_fichier(self, chemin_fichier, encoding='utf-8'):
        """Analyser un fichier texte pour extraire les SIREN"""
        try:
            with open(chemin_fichier, 'r', encoding=encoding) as f:
                texte = f.read()
            
            siren_valides, siren_invalides = self.extraire_tous_siren(texte)
            
            return {
                'fichier': chemin_fichier,
                'siren_valides': siren_valides,
                'siren_invalides': siren_invalides,
                'nb_valides': len(siren_valides),
                'nb_invalides': len(siren_invalides)
            }
        
        except FileNotFoundError:
            print(f"Erreur: Le fichier {chemin_fichier} n'existe pas.")
            return None
        except UnicodeDecodeError:
            print(f"Erreur d'encodage. Tentative avec latin-1...")
            try:
                with open(chemin_fichier, 'r', encoding='latin-1') as f:
                    texte = f.read()
                siren_valides, siren_invalides = self.extraire_tous_siren(texte)
                return {
                    'fichier': chemin_fichier,
                    'siren_valides': siren_valides,
                    'siren_invalides': siren_invalides,
                    'nb_valides': len(siren_valides),
                    'nb_invalides': len(siren_invalides)
                }
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier: {e}")
                return None
    
    def sauvegarder_resultats_csv(self, resultats, dossier_sortie="resultats"):
        """Sauvegarder les résultats dans un fichier CSV"""
        if not resultats:
            print("Aucun résultat à sauvegarder.")
            return None
        
        # Créer le dossier de sortie
        os.makedirs(dossier_sortie, exist_ok=True)
        
        # Préparer les données
        donnees = []
        for i, siren in enumerate(resultats['siren_valides'], 1):
            donnees.append({
                'siren': siren,
                'valide': True,
                'fichier_source': os.path.basename(resultats['fichier'])
            })
        
        for i, siren in enumerate(resultats['siren_invalides'], len(resultats['siren_valides']) + 1):
            donnees.append({
                'siren': siren,
                'valide': False,
                'fichier_source': os.path.basename(resultats['fichier'])
            })
        
        # Créer le DataFrame et sauvegarder
        df = pd.DataFrame(donnees)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        nom_fichier_csv = f"{dossier_sortie}/siren_extraits_{timestamp}.csv"
        df.to_csv(nom_fichier_csv, index=False, encoding='utf-8')
        
        print(f"Résultats sauvegardés dans: {nom_fichier_csv}")
        return nom_fichier_csv
    def sauvegarder_resultats_json(self, resultats, dossier_sortie="resultats"):
        """Sauvegarder les résultats dans un fichier JSON"""
        if not resultats:
            print("Aucun résultat à sauvegarder.")
            return None

        # Créer le dossier de sortie
        os.makedirs(dossier_sortie, exist_ok=True)

        # Préparer les données au format JSON
        donnees = {
            "fichier_source": os.path.basename(resultats["fichier"]),
            "date_extraction": datetime.now().isoformat(),
            "siren_valides": [
                {"siren": s, "valide": True} for s in resultats["siren_valides"]
            ],
            "siren_invalides": [
                {"siren": s, "valide": False} for s in resultats["siren_invalides"]
            ]
        }

        # Sauvegarder dans un fichier JSON
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        nom_fichier_json = f"{dossier_sortie}/siren_extraits_{timestamp}.json"
        
        with open(nom_fichier_json, "w", encoding="utf-8") as f:
            json.dump(donnees, f, indent=4, ensure_ascii=False)

        print(f"Résultats sauvegardés dans : {nom_fichier_json}")
        return nom_fichier_json
    def afficher_resultats(self, resultats):
        """Afficher un résumé des résultats"""
        if not resultats:
            print("Aucun résultat à afficher.")
            return
        
        print(f"\n=== RÉSULTATS D'EXTRACTION SIREN ===")
        print(f"Fichier analysé: {resultats['fichier']}")
        print(f"SIREN valides trouvés: {resultats['nb_valides']}")
        print(f"SIREN invalides trouvés: {resultats['nb_invalides']}")
        
        if resultats['siren_valides']:
            print(f"\nSIREN valides:")
            for siren in resultats['siren_valides']:
                print(f"  - {siren}")
        
        if resultats['siren_invalides']:
            print(f"\nSIREN invalides (échec validation):")
            for siren in resultats['siren_invalides']:
                print(f"  - {siren}")


def main():
    """Fonction principale pour utiliser l'extracteur SIREN"""
    # Initialiser l'extracteur
    extracteur = SirenExtractor()
    
    # Chemin du fichier à analyser (modifiez selon vos besoins)
    chemin_fichier = "D:/Bureau/Stage La gazette/projet ia n°de siren/annonces légales/test annonce légale saone et loire 0206.txt"
    
    # Analyser le fichier
    print("Analyse du fichier en cours...")
    resultats = extracteur.analyser_fichier(chemin_fichier)
    
    if resultats:
        # Afficher les résultats
        extracteur.afficher_resultats(resultats)
        
        # Sauvegarder en CSV
        fichier_csv = extracteur.sauvegarder_resultats_csv(resultats)

        # Sauvegarde en JSON
        fichier_JSON = extracteur.sauvegarder_resultats_json(resultats)

        print(f"\n✅ Exctraction terminée avec succès!")
        
    else:
        print("❌ Erreur lors de l'analyse du fichier.")
    
    # Lancer l'extraction principale
main()

Analyse du fichier en cours...

=== RÉSULTATS D'EXTRACTION SIREN ===
Fichier analysé: D:/Bureau/Stage La gazette/projet ia n°de siren/annonces légales/test annonce légale saone et loire 0206.txt
SIREN valides trouvés: 32
SIREN invalides trouvés: 0

SIREN valides:
  - 818467615
  - 884297169
  - 507384683
  - 888617651
  - 889671509
  - 513603472
  - 850847955
  - 931808331
  - 891748444
  - 893609776
  - 685550659
  - 404146656
  - 980904890
  - 833143662
  - 499901205
  - 915366520
  - 929199701
  - 451132534
  - 531997294
  - 887913424
  - 808775639
  - 518776562
  - 750626269
  - 750583486
  - 847815826
  - 483629036
  - 317587038
  - 970506812
  - 948790381
  - 840026298
  - 822894770
  - 851006817
Résultats sauvegardés dans: resultats/siren_extraits_20250612_105258.csv
Résultats sauvegardés dans : resultats/siren_extraits_20250612_105258.json

✅ Exctraction terminée avec succès!
